# Python for (open) Neuroscience

_Lecture 3.3_ - Running experiments

Luigi Petrucco

Jean-Charles Mariani

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec/blob/main/lectures/Lecture3.3_Run-experiments.ipynb)

## Outline

 - The basics:
     - timing
     - APIs
 - Acquiring data:
     - cameras
     - Arduino
     - Other devices
 - Generating stimuli:
     - Psychopy

## The basics: the idea of a live application

When we are analysing data, we have a script that just runs once (ideally, as fast as possible).

For both acquiring data and control stimuli, we need a live application: something that keeps running until we shut it down or the experiment is finished

### The event loop

To keep the program running, often we define an <span style="color:indianred">event loop</span>! Something that might look like:

In [ ]:
while True:
    pass

### `time` in Python

To control more precisely what happens in event loops we need tools to check timing of the application.

The `time` module (from the standard library) offers such tools!

In [ ]:
import time
time.time()  # returns a float, seconds since January 1, 1970, 00:00:00 ("Unix epoch")

In [ ]:
TOTAL_TIME = 3

print("Program starts")
start_time = time.time()  
while time.time() - start_time < TOTAL_TIME:
    pass

print("Program ends")

Now we control time, but there's an undefined amount of iterations in our program:

In [ ]:
TOTAL_TIME = 1
N_REPS_TEST = 3

for _ in range(N_REPS_TEST):
    counter = 0
    start_time = time.time() 
    
    while time.time() - start_time < TOTAL_TIME:
        counter += 1

    print(f"Program ends, after {counter} iterations (avg. {TOTAL_TIME*10**9/(counter+1)} ns/iteration)")

The execution speed can fluctuate a lot. It depends on both the Python code we write and the availability of cpu resources!

We can control more precisely the loop event timing using `sleep` to pause the program in a controlled fashon:

In [ ]:
PAUSE_S = 0.001

for _ in range(N_REPS_TEST):
    counter = 0
    start_time = time.time() 
    
    while time.time() - start_time < TOTAL_TIME:
        time.sleep(PAUSE_S)
        counter += 1

    print(f"Program ends, after {counter} iterations (avg. {TOTAL_TIME*10**3/(counter+1)} ms/iteration)")

Small time intervals with `sleep` are not crazy good! 
 - OS dependent - should always be tested
 - reliable down to tens of ms, below more fluctuations expected
 - If you need sub-ms accuracy in your application maybe Python is not the way to go!

There are some tricks that can be used to control more tightly access to the cpu, resulting in better timing. 

E.g., `psychopy` `wait()` function:

In [ ]:
from psychopy import clock

PAUSE_S = 0.001

for _ in range(N_REPS_TEST):
    counter = 0
    start_time = time.time() 
    
    while time.time() - start_time < TOTAL_TIME:
        clock.wait(PAUSE_S)
        counter += 1

    print(f"Program ends, after {counter} iterations (avg. {TOTAL_TIME*10**3/(counter+1)} ms/iteration)")

## Measure time

We can also use `time()` to measure elapsed time (e.g., a reaction time):

In [ ]:
time.sleep(1)  # pause 1 second before starting the task
start = time.time()
_ = input("Press enter!")  # a new function to pass inputs to a script! 
end = time.time()
print(f"Reaction time: {end - start} s") 

## The basics: APIs

Many pieces of hardware come with their Python Application Programming Interface (API).

The API of a piece of harwardare is a package that we pip install and we can use to control the hardware

Usually an object-oriented interface! The piece of hardware is represented very naturally with an object.

## Acquiring data

### Example: cameras

In [ ]:
# pip install python-opencv to use:
import cv2
from matplotlib import pyplot as plt
 
# Instantiate a video capture object (camera):
vid = cv2.VideoCapture(0)

In [ ]:
state, frame = vid.read()  # read method to capture frame

In [ ]:
frame.shape

In [ ]:
plt.figure(figsize=(3, 3))
plt.imshow(frame[:, :, [2,1,0]])  # Frame is BGR instead of RGB, swap colors to plot

After we're done with a piece of hardware, it is always good to shut it off:

In [ ]:
vid.release()  # close the camera reader and free to camera

Many cameras from many different vendors offer APIs!

(Practical 3.3.0)

## Something on electronics

We read data and control hardware mostly using:
 - digital signals
 - analog signals

### Analog signals

A continuous voltage in a range (usually 0-5 V or 0-10 V) maps some continuous variable.

Analog signals are acquired and generated by specialized hardware - your computer can't talk analog with any kind of device!

That's part of the reason why we use Arduinos, NI boards, digitizers, etc.

### Digital signals

Discrete signals - ON or OFF (usually 0/3.3 V or 0/5 V, depending on the device)

Much more robust to noise! Those are the signals that digital devices such as the computer use

As fast as analog signals, but with very small bandwidth (two states, ON/OFF)

### Serial communication

Digital signals can be used to transmit bits of information through time!

- This requires the sender and the receiver to agree on time (baud rate)


# Arduino

- Open-source microcontroller
- Has digital/analogic input and output channels

## What is an Arduino

### Arduino code

### Serial communication

### Firmata language

## Acquiring digital and analogic signals

## Producing digital and analogic signals

## Running experiments in Python

**Pros**

- Jack-of-all-trades: we need to remember only one language for experiments, preprocessing, stats, etc.
- We can save and load data with the same libraries
- Thousands of available libraries
- Thousands of available APIs

**Cons**

- Execution time becomes unreliable at the ms/sub-ms scale (unless we use specialized hardware)
- Concurrency issues

## Timing events

# Saving sata